In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [30]:
# init
steepness = 50    # steepness factor
num_blocks = 2    # number of expression blocks
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [31]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [32]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [33]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    model.set_param_val({"p":p, "a":a})
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    params.append(list(p)+list(a))
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_exact_m.csv")

  5%|████▏                                                                         | 53/1000 [06:29<1:55:56,  7.35s/it]

KeyboardInterrupt



## Heuristic

In [34]:
model_heur = model.first_solution_heuristic()

In [35]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    model_heur.set_param_val({"p":p, "a":a})
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    params.append(list(p)+list(a))
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_m.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:25<00:00, 11.72it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean    950.369241               0.0      0.084850
std     616.480055               0.0      0.039165
min     119.733136               0.0      0.059156
25%     321.880600               0.0      0.062185
50%     806.734458               0.0      0.075879
75%    1499.065030               0.0      0.078666
max    2022.924515               0.0      0.388860
Number of infeasible solution: 0


## Learnable Rounding

In [36]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 32            # width of hidden layers for solution mapping
lr = 1e-2             # learning rate
batch_size = 64       # batch size

In [37]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# define Rosenbrock objective functions and constraints for both problem types
obj_rel, constrs_rel = nmRosenbrock(["x"], ["p", "a"], steepness, num_blocks, penalty_weight=penalty_weight)
obj_rnd, constrs_rnd = nmRosenbrock(["x_rnd"], ["p", "a"], steepness, num_blocks, penalty_weight=penalty_weight)
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],
                       output_keys=["x_rnd"], int_ind=model.int_ind, name="round")
# build neuromancer problem for rounding
components = [smap, rnd]
loss = nm.loss.PenaltyLoss(obj_rnd, constrs_rnd)
problem = nm.problem.Problem(components, loss)

In [38]:
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.Adam(problem.parameters(), lr=lr)
# create a trainer for the problem
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test, optimizer, 
                             epochs=epochs, patience=patience, warmup=warmup)
# training for the rounding problem
best_model = trainer.train()
# load best model dict
problem.load_state_dict(best_model)

epoch: 0  train_loss: 104.50736236572266
epoch: 1  train_loss: 49.16930389404297
epoch: 2  train_loss: 44.45543670654297
epoch: 3  train_loss: 35.22999954223633
epoch: 4  train_loss: 34.56815719604492
epoch: 5  train_loss: 27.8295955657959
epoch: 6  train_loss: 28.724287033081055
epoch: 7  train_loss: 31.84865951538086
epoch: 8  train_loss: 27.3794002532959
epoch: 9  train_loss: 23.089242935180664
epoch: 10  train_loss: 20.871292114257812
epoch: 11  train_loss: 18.80219078063965
epoch: 12  train_loss: 19.508909225463867
epoch: 13  train_loss: 17.975379943847656
epoch: 14  train_loss: 18.356740951538086
epoch: 15  train_loss: 17.13743782043457
epoch: 16  train_loss: 18.1928653717041
epoch: 17  train_loss: 13.30143928527832
epoch: 18  train_loss: 13.790485382080078
epoch: 19  train_loss: 12.904794692993164
epoch: 20  train_loss: 14.5535306930542
epoch: 21  train_loss: 12.94597339630127
epoch: 22  train_loss: 12.660889625549316
epoch: 23  train_loss: 15.191707611083984
epoch: 24  train_lo

<All keys matched successfully>

In [39]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32),
                  "name": "test"}
    # infer
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = output["test_x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_nm_m.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 304.04it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     13.494995          0.000381      0.002816
std       8.372295          0.006183      0.000833
min       0.172183          0.000000      0.001000
25%       7.683434          0.000000      0.002006
50%      11.968182          0.000000      0.002996
75%      17.553032          0.000000      0.003018
max      45.346165          0.134142      0.007729
Number of infeasible solution: 4
